In [43]:
import pandas as pd

# read in

In [44]:
df = pd.read_csv('../../2_Feature_Engineering/export/combined_polly_bretschneider_iwg.csv')
df.head()

,raw_text,severity,binary_label
0,ist ekelhaft bei ihren Klos kannst du nur mit...,0.0,False
1,Bevor Sie mit Jud...,1.0,True
2,Und de Amis freut es wenn die Heuschrecken Eur...,0.0,False
3,": Die Krim ist ein heißer Tip, auch zum In...",1.0,True
4,: Es muss in #Deutschland eine politische Kra...,1.0,True


# clean

In [45]:
import sys
sys.path.insert(0, r'..\..\0_common')
from model_helpers import clean_all as ci

In [46]:
df['cleaned_text'] = df['raw_text'].apply(ci)
df.head()

,raw_text,severity,binary_label,cleaned_text
0,ist ekelhaft bei ihren Klos kannst du nur mit...,0.0,False,ekelhaft klos kannst gummistiefel gehen stinkt...
1,Bevor Sie mit Jud...,1.0,True,bevor judenstern markiert worden juden ganze ...
2,Und de Amis freut es wenn die Heuschrecken Eur...,0.0,False,de amis freut heuschrecken europa klein wirtsc...
3,": Die Krim ist ein heißer Tip, auch zum In...",1.0,True,krim heißer tip investieren muß bürger landes...
4,: Es muss in #Deutschland eine politische Kra...,1.0,True,deutschland politische kraft geben sozial sch...


In [47]:
df.shape

(9169, 4)

# vectorize

In [48]:
from sklearn.feature_extraction.text import CountVectorizer

In [49]:
vec = CountVectorizer(ngram_range=(1,3), analyzer='word', max_features=100000)

In [50]:
X_features = vec.fit_transform(df['cleaned_text'])

In [51]:
X_features.shape

(9169, 100000)

# initialize model

In [52]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(solver='liblinear', penalty='l2', C=0.23357214690901212) # params from earlier grid search

# Cross validation

In [53]:
from sklearn.model_selection import cross_validate
scores = cross_validate(clf, X_features, df['binary_label'], cv=5,
                        scoring=('f1', 'roc_auc', 'precision', 'recall'),
                        return_train_score=True)

In [54]:
sorted(scores.keys())

['fit_time',
 'score_time',
 'test_f1',
 'test_precision',
 'test_recall',
 'test_roc_auc',
 'train_f1',
 'train_precision',
 'train_recall',
 'train_roc_auc']

In [55]:
scores_df = pd.DataFrame(scores)
scores_df

,fit_time,score_time,test_f1,train_f1,test_roc_auc,train_roc_auc,test_precision,train_precision,test_recall,train_recall
0,0.110703,0.004987,0.866343,0.977582,0.945404,0.998524,0.911765,0.989911,0.825231,0.965557
1,0.094747,0.004986,0.870050,0.976355,0.947494,0.998607,0.934840,0.991055,0.813657,0.962084
2,0.083776,0.003990,0.855028,0.974879,0.945528,0.998558,0.915456,0.989857,0.802083,0.960347
3,0.074806,0.005979,0.876308,0.975015,0.956705,0.998404,0.935611,0.990445,0.824074,0.960058
4,0.096741,0.005984,0.880000,0.977680,0.955202,0.998589,0.922490,0.992546,0.841251,0.963252


In [56]:
# metrics
scores_df[['test_f1', 'test_roc_auc', 'test_precision', 'test_recall']].mean()

test_f1           0.869546
test_roc_auc      0.950067
test_precision    0.924032
test_recall       0.821260
dtype: float64